## Importing necessary packages


In [ ]:
# Numerical Operations
import math
import numpy as np

# Reading/Writing Data
import pandas as pd
import os
import csv

# For Progress Bar
from tqdm import tqdm

# Pytorch
import torch 
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split

# For plotting learning curve
from torch.utils.tensorboard import SummaryWriter

## Loading data

In [ ]:
# mnist_trainset = datasets.MNIST('/content/drive/MyDrive/Machine Learning for Smart Space-Based Radiation and X-Ray Spectrometer Instruments/Code/Preprocessing/Lu_integrals.csv', train=True, download=True, 
#                                 transform=transforms.ToTensor())
# mnist_testset = datasets.MNIST('/content/drive/MyDrive/Machine Learning for Smart Space-Based Radiation and X-Ray Spectrometer Instruments/Code/Preprocessing/Lu_integrals.csv', train=False, download=True, 
#                                transform=transforms.ToTensor())
# print("Print the training dataset:\n ", mnist_trainset)
# print("Print the testing dataset:\n ", mnist_testset)


train_data = pd.read_csv("/content/drive/MyDrive/Machine Learning for Smart Space-Based Radiation and X-Ray Spectrometer Instruments/Code/Preprocessing/Lu_integrals.csv")
#test_data = pd.read_csv("hw2_p1_test_data.csv")
train_data.head()

,3.949949737136349185e-09
0,2.221671e-09
1,3.665683e-09
2,9.885014e-09
3,2.527140e-09
4,1.581143e-09


## Selecting the features we want

In [ ]:
# def select_feat(train_data, valid_data, test_data, select_all=True):
#     '''Selects useful features to perform regression'''
#     y_train, y_valid, y_test = train_data[:,0], valid_data[:,0], test_data[:,0]
#     raw_x_train, raw_x_valid, raw_x_test = train_data[:,1:], valid_data[:,1:], test_data[:,1:]

#     if select_all:
#         feat_idx = list(range(raw_x_train.shape[1]))
#     else:
#         feat_idx = [0,1,2,3,4] # TODO: Select suitable feature columns, but we are using MNIST, so better select all
        
    # return raw_x_train[:,feat_idx], raw_x_valid[:,feat_idx], raw_x_test[:,feat_idx], y_train, y_valid, y_test

## Training Loop

In [ ]:
def trainer(train_loader, valid_loader, model, config, device):

    criterion = nn.CrossEntropyLoss(reduction='mean') # Define your loss function, do not modify this.

    # Define your optimization algorithm. 
    # TODO: Please check https://pytorch.org/docs/stable/optim.html to get more available algorithms.
    optimizer = torch.optim.Adam(model.parameters(), lr=config['learning_rate']) 
    writer = SummaryWriter() # Writer of tensoboard.

    if not os.path.isdir('./models'):
        os.mkdir('./models') # Create directory of saving models.

    n_epochs, best_loss, step, early_stop_count = config['n_epochs'], math.inf, 0, 0

    for epoch in range(n_epochs):
        model.train() # Set your model to train mode.
        loss_record = []

        # tqdm is a package to visualize your training progress.
        train_pbar = tqdm(train_loader, position=0, leave=True)

        for x, y in train_pbar:
            optimizer.zero_grad()               # Set gradient to zero.
            x, y = x.to(device), y.to(device)   # Move your data to device. 
            pred = model(x)
            loss = criterion(pred, y)
            loss.backward()                     # Compute gradient(backpropagation).
            optimizer.step()                    # Update parameters.
            step += 1
            loss_record.append(loss.detach().item())
            
            # Display current epoch number and loss on tqdm progress bar.
            train_pbar.set_description(f'Epoch [{epoch+1}/{n_epochs}]')
            train_pbar.set_postfix({'loss': loss.detach().item()})

        mean_train_loss = sum(loss_record)/len(loss_record)
        writer.add_scalar('Loss/train', mean_train_loss, step)

        model.eval() # Set your model to evaluation mode.
        loss_record = []
        val_accuracy = []
        for x, y in valid_loader:
            x, y = x.to(device), y.to(device)
            with torch.no_grad():
                pred = model(x)
                loss = criterion(pred, y)

                _, predicted = torch.max(pred.data, 1)
                val_accuracy.append((predicted == y).sum().item() / predicted.size(0))

            loss_record.append(loss.item())
            
        mean_valid_loss = sum(loss_record)/len(loss_record)
        overall_val_accuracy = np.mean(val_accuracy)
        print(f'Epoch [{epoch+1}/{n_epochs}]: Train loss: {mean_train_loss:.4f}, Valid loss: {mean_valid_loss:.4f} Valid acc: {overall_val_accuracy:.4f}')
        # writer.add_scalar('Loss/valid', mean_valid_loss, step)

        if mean_valid_loss < best_loss:
            best_loss = mean_valid_loss
            torch.save(model.state_dict(), config['save_path']) # Save your best model
            print('Saving model with loss {:.3f}...'.format(best_loss))
            early_stop_count = 0
        else: 
            early_stop_count += 1

        if early_stop_count >= config['early_stop']:
            print('\nModel is not improving, so we halt the training session.')
            return

## Configuration of our model

In [ ]:
def same_seed(seed): 
    '''Fixes random number generator seeds for reproducibility.'''
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
config = {
    'seed': 5201314,      # Your seed number, you can pick your lucky number. :)
    'select_all': True,   # Whether to use all features.
    'valid_ratio': 0.2,   # validation_size = train_size * valid_ratio
    'n_epochs': 100,     # Number of epochs.            
    'batch_size': 256, 
    'learning_rate': 0.00001,              
    'early_stop': 100,    # If model has not improved for this many consecutive epochs, stop training.     
    'save_path': './models/model.ckpt'  # Your model will be saved here.
}

## Setting up the training, validation, and testing sets

In [ ]:
class RadiationDataset(Dataset):
    def __init__(self, x, y=None):
        if y is None:
            self.y = None
        else:
            self.y = torch.LongTensor(y)
        self.x = torch.FloatTensor(x)

    def __getitem__(self, idx):
        if self.y is None:
            return self.x[idx]
        else:
            return self.x[idx], self.y[idx]

    def __len__(self):
        return len(self.x)

In [ ]:
def train_valid_split(data_set, valid_ratio, seed):
    '''Split provided training data into training set and validation set'''
    valid_set_size = int(valid_ratio * len(data_set)) 
    train_set_size = len(data_set) - valid_set_size
    train_set, valid_set = random_split(data_set, [train_set_size, valid_set_size], generator=torch.Generator().manual_seed(seed))
    return np.array(train_set), np.array(valid_set)

In [ ]:
same_seed(config['seed'])
train_data, test_data = pd.read_csv("/content/drive/MyDrive/Machine Learning for Smart Space-Based Radiation and X-Ray Spectrometer Instruments/Code/Preprocessing/Lu_integrals.csv").values, pd.read_csv("/content/drive/MyDrive/Machine Learning for Smart Space-Based Radiation and X-Ray Spectrometer Instruments/Code/Preprocessing/Lu_integrals.csv").values
train_data, valid_data = train_valid_split(train_data, config['valid_ratio'], config['seed'])

# Print out the data size.
print(f"""train_data size: {train_data.shape} 
valid_data size: {valid_data.shape} 
test_data size: {test_data.shape}""")

# Select features
x_train, x_valid, x_test, y_train, y_valid, y_test = select_feat(train_data, valid_data, test_data, config['select_all'])

# Print out the number of features.
print(f'number of features: {x_train.shape[1]}')

# x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.2, random_state=0)
train_dataset, valid_dataset, test_dataset = RadiationDataset(x_train, y_train), \
                                             RadiationDataset(x_valid, y_valid), \
                                             RadiationDataset(x_test)

# Pytorch data loader loads pytorch dataset into batches.
train_loader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True, pin_memory=True)
valid_loader = DataLoader(valid_dataset, batch_size=config['batch_size'], shuffle=True, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=config['batch_size'], shuffle=False, pin_memory=True)

train_data size: (8799, 1) 
valid_data size: (2199, 1) 
test_data size: (10998, 1)
number of features: 0


In [ ]:
print(train_loader)

## Actual training

In [ ]:
class My_Model(nn.Module):
    def __init__(self, input_dim):
        super(My_Model, self).__init__()
        # TODO: modify model's structure, be aware of dimensions. 
        self.layers = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 10)
        )

    def forward(self, x):
        x = self.layers(x)
        # x = x.squeeze(1) # (B, 1) -> (B)
        return x

In [ ]:
model = My_Model(input_dim=x_test.shape[1]).to(device) # put your model and data on the same computation device.
trainer(train_loader, valid_loader, model, config, device)

/usr/local/lib/python3.10/dist-packages/torch/nn/init.py:405: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")
Epoch [1/100]: 100%|██████████| 35/35 [00:00<00:00, 93.08it/s, loss=2.19]


Epoch [1/100]: Train loss: 2.1862, Valid loss: 2.1855 Valid acc: 1.0000
Saving model with loss 2.186...


Epoch [2/100]: 100%|██████████| 35/35 [00:00<00:00, 96.39it/s, loss=2.18]


Epoch [2/100]: Train loss: 2.1849, Valid loss: 2.1842 Valid acc: 1.0000
Saving model with loss 2.184...


Epoch [3/100]: 100%|██████████| 35/35 [00:00<00:00, 96.01it/s, loss=2.18]


...

Epoch [98/100]: Train loss: 2.0403, Valid loss: 2.0394 Valid acc: 1.0000
Saving model with loss 2.039...


Epoch [99/100]: 100%|██████████| 35/35 [00:00<00:00, 82.81it/s, loss=2.04]


Epoch [99/100]: Train loss: 2.0385, Valid loss: 2.0377 Valid acc: 1.0000
Saving model with loss 2.038...


Epoch [100/100]: 100%|██████████| 35/35 [00:00<00:00, 84.66it/s, loss=2.04]


Epoch [100/100]: Train loss: 2.0368, Valid loss: 2.0359 Valid acc: 1.0000
Saving model with loss 2.036...


In [ ]:

def predict(test_loader, model, device):
    model.eval() # Set your model to evaluation mode.
    preds = []
    for x in tqdm(test_loader):
        x = x.to(device)                        
        with torch.no_grad():                   
            pred = model(x)                     
            preds.append(pred.detach().cpu())   
    preds = torch.cat(preds, dim=0).numpy()  
    return preds

In [ ]:
model = My_Model(input_dim=x_train.shape[1]).to(device)
model.load_state_dict(torch.load(config['save_path']))
preds = predict(test_loader, model, device)



test_accuracy = []
pred = torch.Tensor(preds)
y = torch.Tensor(y_test)
pred, y = pred.to(device), y.to(device)
with torch.no_grad():
    _, predicted = torch.max(pred, 1)
    test_accuracy.append((predicted == y).sum().item() / predicted.size(0))
    
overall_val_accuracy = np.mean(test_accuracy)
print('overall test accuracy:', overall_val_accuracy)


100%|██████████| 43/43 [00:00<00:00, 947.66it/s]

overall test accuracy: 0.0
